We successfully wrote a program for solving sudoku....Even though it is iterative(until decision making comes into picture) and backtracking(bruteforce with backtracking) from decision making onwards.....next see how others have written programs for sudoku and learn from them...and search for better approaches

Expert Example :
0 8 0 0 0 0 0 9 0 0 7 5 3 6 0 0 0 8 0 0 0 0 4 0 0 0 0 6 5 0 0 0 2 3 0 0 0 0 0 4 5 0 0 0 0 2 0 9 0 0 0 0 0 0 0 0 0 0 0 7 5 0 0 0 0 0 0 1 3 6 0 0 1 0 0 0 0 0 0 2 0 

In [288]:
import numpy as np

In [289]:
# give your input in one line(i.e., in oneline give the 81 numbers space seperated), give 0's for the empty spaces in the sudoku and then press Enter
# eg:easy sudoku : 0 3 4 8 7 0 0 0 0 0 0 2 3 4 0 1 5 8 1 8 0 2 0 0 0 0 0 9 0 0 0 3 0 5 0 6 0 1 0 0 0 0 4 0 0 0 5 3 7 6 0 0 0 2 0 2 0 6 0 3 0 0 0 7 0 8 0 0 5 0 0 3 3 0 1 4 0 7 8 6 5
arr=list(map(int,input().split(" ")))
print(len(arr))

0 8 0 0 0 0 0 9 0 0 7 5 3 6 0 0 0 8 0 0 0 0 4 0 0 0 0 6 5 0 0 0 2 3 0 0 0 0 0 4 5 0 0 0 0 2 0 9 0 0 0 0 0 0 0 0 0 0 0 7 5 0 0 0 0 0 0 1 3 6 0 0 1 0 0 0 0 0 0 2 0
81


In [290]:
set1={0,1,2}
set2={3,4,5}
set3={6,7,8}
def get_a(i):
    if(i in set1):
        a=0
    elif(i in set2):
        a=3
    elif(i in set3):
        a=6
    return a
def get_b(j):
    if(j in set1):
        b=0
    elif(j in set2):
        b=1
    elif(j in set3):
        b=2
    return b
def get_box_index(i,j):
    
    a=get_a(i)
    b=get_b(j)
    return a+b
    
    
def create_sets(arr,n,nine_set):
    # create_sets create different sets like sets_matrix(n*n board where each element is a set containing what are the choices we have at that place)
    # rows is a 9 element list where each is a set which contains all the elements present in that rows
    # cols is a 9 element list where each is a set which contains all the elements present in that cols
    # boxes is a 9 element list where each is a set which contains all the elements present in that box(we have 9 boxes in a sudoku)
    sets=[set() for i in range(n*n)]
    sets_matrix=np.array(sets).reshape(n,n)
    rows=[set() for i in range(n)]
    cols=[set() for i in range(n)]
    boxes=[set() for i in range(n)]
    for i in range(n):
        for j in range(n):
            if(arr[i][j] != 0):
                rows[i].add(arr[i][j])
                cols[j].add(arr[i][j])
                boxes[get_box_index(i,j)].add(arr[i][j]) #get_box_index takes the i,j indices and returns the box number(0-8)
    for i in range(n):
        for j in range(n):
            if(arr[i][j] == 0):
                sets_matrix[i][j]=sets_matrix[i][j].union(nine_set.difference(boxes[get_box_index(i,j)].union(rows[i].union(cols[j]))))
    return sets_matrix,rows,cols,boxes
                
def unique_check(i,j,sets_matrix):
    # This function checks whether there is only one choice at a place (eg: {3} at a place in sets_matrix means 
    # 3 is the only choice at that place so we will place 3 at that place)
    if(len(sets_matrix[i][j])==1):
        return True
    return False
def horizontal_unique_check(i,j,n,sets_matrix):
    # For a given place (i,j) this function checks whether there is any element in that place which can only occur
    # in that place(in sets_matrix) and not occured in any other place(in sets_matrix) horizontally 
    temp=sets_matrix[i][j].copy()
    for k in range(n):
        if(k!=j):
            temp=temp.difference(sets_matrix[i][k])
    if(len(temp)==1):
        for x in temp:
            return True,x
    return False,0
def vertical_unique_check(i,j,n,sets_matrix):
    # For a given place (i,j) this function checks whether there is any element in that place which can only occur
    # in that place(in sets_matrix) and not occured in any other place(in sets_matrix) vertically
    temp=sets_matrix[i][j].copy()
    for k in range(n):
        if(k!=i):
            temp=temp.difference(sets_matrix[k][j])
    if(len(temp)==1):
        for x in temp:
            return True,x
    return False,0
def box_unique_check(i,j,n,sets_matrix):
    # For a given place (i,j) this function checks whether there is any element in that place which can only occur
    # in that place(in sets_matrix) and not occured in any other place(in sets_matrix) in that box which (i,j) belongs to
    temp=sets_matrix[i][j].copy()
    for p in range(get_a(i),get_a(i)+3):
        for q in range(get_b(j)*3,get_b(j)*3+3):
            if(p!=i or q!=j):
                temp=temp.difference(sets_matrix[p][q])
    if(len(temp)==1):
        for x in temp:
            return True,x
    return False,0      
def check_isover(arr,n):
    # Return True if game over else False 
    over=True
    for i in range(n):
        for j in range(n):
            if(arr[i][j]==0):
                over=False
    return over
def no_of_choices_for_each_element(sets_matrix,n):
    n_sets_matrix=[]
    for i in range(n):
        for j in range(n):
            if(len(sets_matrix[i][j])==0):
                n_sets_matrix.append(9)
            else:
                n_sets_matrix.append(len(sets_matrix[i][j]))
    n_sets_matrix=np.array(n_sets_matrix).reshape(n,n)
    return n_sets_matrix
def get_indices(flat,n):
    # for a flat index in a matrix it gives the (i,j) values 
    for j in range(0,n):
        if((flat-j)%n==0):
            q=j
            break
    p=(flat-j)/n
    return int(p),q
def hv_mistake_test(arr,n):
    # It checks whether there is any clash horizontally or there is any clash vertically
    # (i.e same element occurs at 2 places in the same row or same col) 
    for i in range(n):
        h=[]
        v=[]
        for j in range(n):
            if(arr[i][j]!=0 ):
                if(arr[i][j] not in h):
                    h.append(arr[i][j])
                else:
                    return True
            if(arr[j][i]!=0):
                if(arr[j][i] not in v):
                    v.append(arr[j][i])
                else:
                    return True
    #print("I'm in hv and returning false")
    return False
def box_mistake_test(arr):
    # It checks whether there is a clash in any box
    # (i.e same element occurs at 2 places in the same box) 
    for i in [0,3,6]:
        for j in [0,3,6]:
            b=[]
            for p in range(i,i+3):
                for q in range(j,j+3):
                    if(arr[p][q]!=0):
                        if(arr[p][q] not in b):
                            b.append(arr[p][q])
                        else:
                            return True
    return False
                    
            
def mistake_test(arr,n):
    # return true if there is a mistake else false
    if(hv_mistake_test(arr,n) or box_mistake_test(arr)):
        return True
    return False
        
def isall_nines(n_sets_matrix,n):
    # this is used because some times at some point during recursion at some places the element it is not filled(indicated with 0)
    # and also have no choices to make(sets_matrix at the place is empty)
    # so in order to address this issue we wrote this function the purpose of this function is 
    # "since i am calling this function after checking the game is over that means the game is not over and n_sets_matrix denotes 
    # the number of choices at a place(if the place is filled we denote it with 9 in n_sets_matrix(serves for the purpose of argmin 
    # for reducing the breadth of the decision tree(during recursion))) so all places should not be 9's...
    # if all places are 9's there is some errorr so we traceback
    for i in range(n):
        for j in range(n):
            if(n_sets_matrix[i][j]!=9):
                return False
    return True
                
    
def recursive_backtracking_approach(arr,sets_matrix,n,nine_set,n_sets_matrix):
    if(mistake_test(arr,n)):
        return arr,False
    elif(check_isover(arr,n)):
        print("SUDOKU COMPLETED")
        print(arr)
        return arr,True
    elif(isall_nines(n_sets_matrix,n)):
        return arr,False
    else:
        arr_copy=arr.copy()
        flat=np.argmin(n_sets_matrix)
        p,q=get_indices(flat,n)
        #print(p,q)
        arr_copy[p][q]=list(sets_matrix[p][q])[0]
        sets_matrix_copy,_,_,_=create_sets(arr_copy,n,nine_set)
        n_sets_matrix_copy=no_of_choices_for_each_element(sets_matrix_copy,n)
        call=0
        while(not recursive_backtracking_approach(arr_copy,sets_matrix_copy,n,nine_set,n_sets_matrix_copy)[1]):
            call+=1
            if(call==len(sets_matrix[p][q])):
                return arr,False
            arr_copy[p][q]=list(sets_matrix[p][q])[call%len(sets_matrix[p][q])]
            sets_matrix_copy,_,_,_=create_sets(arr_copy,n,nine_set)
            n_sets_matrix_copy=no_of_choices_for_each_element(sets_matrix_copy,n)
        return arr,True
        
        
def play_real_sudoku(arr,sets_matrix,rows,cols,boxes,n,nine_set):
    n_sets_matrix=no_of_choices_for_each_element(sets_matrix,n) 
    return recursive_backtracking_approach(arr,sets_matrix,n,nine_set,n_sets_matrix)[0]
    
    
    
    
    

            
    

In [291]:
def initial_push(arr,sets_matrix,rows,cols,boxes,n,nine_set):
    # This function can solve the problems which don't require decisions 
    # (even for the problems which require decisions this will solve till a dicision is required)
    # This will reduce the depth and breadth of the decision tree  
        while(True):
        #     sets_matrix,rows,cols,boxes=create_sets(arr,n,nine_set)
        #     print(sets_matrix)
        #     print(rows)
        #     print(cols)
        #     print(boxes)
        #     print(arr)
            flag=0
            for i in range(n):
                for j in range(n):
                    if(arr[i][j]==0):
                        #print("We are solving for ({i},{j})",i,j)
                        bh,xh=horizontal_unique_check(i,j,n,sets_matrix)
                        bv,xv=vertical_unique_check(i,j,n,sets_matrix)
                        bb,xb=box_unique_check(i,j,n,sets_matrix)
                        if(unique_check(i,j,sets_matrix)):
                            #print("I'm revealed by unique_check")
                            # If a set has only one element then we will put that element in that place
                            flag=1
                            for x in sets_matrix[i][j]:
                                arr[i][j]=x
                                sets_matrix[i][j]=set()
        #                         rows[i].add(x)
        #                         cols[j].add(x)
        #                         boxes[get_box_index(i,j)].add(x)

                        elif(bh):
                            #print("I'm revealed by horizontal_unique_check")
                            flag=1
                            arr[i][j]=xh
                            sets_matrix[i][j]=set()
        #                     rows[i].add(xh)
        #                     cols[j].add(xh)
        #                     boxes[get_box_index(i,j)].add(xh)

                        elif(bv):
                            #print("I'm revealed by vertical_unique_check")
                            flag=1
                            arr[i][j]=xv
                            sets_matrix[i][j]=set()
        #                     rows[i].add(xv)
        #                     cols[j].add(xv)
        #                     boxes[get_box_index(i,j)].add(xv)

                        elif(bb):
                            #print("I'm revealed by box_unique_check")
                            flag=1
                            arr[i][j]=xb
                            sets_matrix[i][j]=set()
        #                     rows[i].add(xb)
        #                     cols[j].add(xb)
        #                     boxes[get_box_index(i,j)].add(xb)
                        sets_matrix,rows,cols,boxes=create_sets(arr,n,nine_set)
                        #print(sets_matrix)
                        #print(arr)
            if(flag==0):
                print(arr)
                if(not check_isover(arr,n)):
                    print("Without decision making we can only solve upto the above board")
                    print("This Problem require decision making")
                    arr=play_real_sudoku(arr,sets_matrix,rows,cols,boxes,n,nine_set)
                    
                    # This arr is useless since in the  recursive_backtracking_approach function i'm not collecting the returned arr
                else:
                    print("SUDOKU COMPLETED")
                    print("This Problem doesn't require any decision making")
                break
            else:
                flag=0

In [292]:
n=9 # for sudoku size 
nine_set=set([1,2,3,4,5,6,7,8,9]) # Later we require a list of 9 elements 
arr=np.array(arr).reshape(n,n) # arr is a n*n board  
sets_matrix,rows,cols,boxes=create_sets(arr,n,nine_set) 
# print(sets_matrix)
# print(rows)
# print(cols)
# print(boxes)
# print(arr)
initial_push(arr,sets_matrix,rows,cols,boxes,n,nine_set)
                
                

[[0 8 0 0 0 0 0 9 0]
 [0 7 5 3 6 0 2 0 8]
 [0 0 0 0 4 0 0 0 0]
 [6 5 0 0 0 2 3 0 0]
 [7 0 0 4 5 0 0 0 2]
 [2 0 9 0 3 0 0 0 0]
 [8 0 0 6 0 7 5 0 0]
 [5 0 0 0 1 3 6 0 0]
 [1 0 0 5 0 4 0 2 0]]
Without decision making we can only solve upto the above board
This Problem require decision making
SUDOKU COMPLETED
[[3 8 2 1 7 5 4 9 6]
 [4 7 5 3 6 9 2 1 8]
 [9 1 6 2 4 8 7 5 3]
 [6 5 1 9 8 2 3 7 4]
 [7 3 8 4 5 1 9 6 2]
 [2 4 9 7 3 6 1 8 5]
 [8 9 4 6 2 7 5 3 1]
 [5 2 7 8 1 3 6 4 9]
 [1 6 3 5 9 4 8 2 7]]
